In [ ]:
# First implementation on Kaggle Notebook
# http://savvastjortjoglou.com/nba-shot-sharts.html


%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # stats model view
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold

In [ ]:
# import data
KobeData= "../input/data.csv"
Kobesub="../input/sample_submission.csv"
raw = pd.read_csv(KobeData)
rawsample = pd.read_csv (Kobesub)

In [ ]:
#read a little sample of KobeData
raw.head()

In [ ]:
#read a little sample of KobeSub
rawsample.head()

In [ ]:
rawtrain= raw[raw['shot_made_flag'].notnull()] 
print("Kobe Data Size", raw.shape)
print("Kobe Train Data Size", rawtrain.shape)
print("Kobe Sub Size", rawsample.shape)

In [ ]:
# creating a basic scatter plot to show the data
alpha=0.2

sns.set_style("white")
sns.set_color_codes()
plt.figure(figsize=(12,11))
plt.scatter(rawtrain['loc_x'],rawtrain['loc_y'], color='#c65b08', alpha=alpha)
# note that x-axis values are the inverse of what they actually should be
# only showing shots up to 50 feet away
plt.xlim(300,-300)
plt.ylim(-100,500)
plt.show()

In [ ]:
#Draw The court

from matplotlib.patches import Circle, Rectangle, Arc

def draw_court (ax=None, color='black', lw=2, outer_lines=False):
    if ax is None:
        ax=plt.gca()
        
    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 18" so it has a radius of 9", which is a value
    # 7.5 in our coordinate system
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)
    
        # Create backboard
    backboard = Rectangle((-30, -7.5), 60, -1, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the side 3pt lines, they are 14ft long before they begin to arc
    corner_three_a = Rectangle((-220, -47.5), 0, 140, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((220, -47.5), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    # I just played around with the theta values until they lined up with the 
    # threes
    three_arc = Arc((0, 0), 475, 475, theta1=22, theta2=158, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)

    # List of the court elements to be plotted onto the axes
    court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                      bottom_free_throw, restricted, corner_three_a,
                      corner_three_b, three_arc, center_outer_arc,
                      center_inner_arc]
    
    #Draw outer_lines 
    if outer_lines : 
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw, color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)
        
    return ax

In [ ]:
plt.figure(figsize=(12,11))
draw_court(outer_lines=True)
plt.scatter(rawtrain['loc_x'],rawtrain['loc_y'], color='#c65b08', alpha=alpha)
plt.xlim(300,-300)
plt.ylim(-100,500)
plt.title('Kobe Shot Loc x, Loc y')
plt.show()

In [ ]:
# create our jointplot

right = rawtrain[rawtrain.shot_zone_area == "Left Side(L)"]
joint_shot_chart = sns.jointplot(right['loc_x'],right['loc_y'], color='#c65b08', stat_func=None,
                                 kind='scatter', space=0, alpha=alpha)





joint_shot_chart.fig.set_size_inches(12,11)

# A joint plot has 3 Axes, the first one called ax_joint 
# is the one we want to draw our court onto and adjust some other settings
ax = joint_shot_chart.ax_joint
draw_court(ax)

# Adjust the axis limits and orientation of the plot in order
# to plot half court, with the hoop by the top of the plot
ax.set_xlim(-250,250)
ax.set_ylim(422.5, -47.5)

# Get rid of axis labels and tick marks
ax.set_xlabel('')
ax.set_ylabel('')
ax.tick_params(labelbottom='off', labelleft='off')

# Add a title
ax.set_title('Kobe Bryant FGA All seasons', 
             y=1.2, fontsize=14)

# Add Data Source 
ax.text(-250,445,'Data Source: Kaggle.com',
        fontsize=12)

plt.show()

In [ ]:
# create our jointplot

cmap=plt.cm.gist_heat_r

#joint_shot_chart = sns.jointplot(rawtrain['loc_x'],rawtrain['loc_y'],stat_func=None,
                              #  kind='hex', space=0, cmap=cmap, color="#4CB391")

g = sns.JointGrid(rawtrain['loc_x'],rawtrain['loc_y'])
g.ax_marg_x.hist(rawtrain['loc_x'], bins=np.arange(-300, 300, 17), color=cmap(0.1))
g.ax_marg_y.hist(rawtrain['loc_y'], bins=np.arange(-47.5,422.5, 20), orientation="horizontal", color=cmap(0.1))
g.plot_joint(plt.hexbin, gridsize=17, extent=[250, -250,422.5, -47.5], cmap="gist_heat_r")



#joint_shot_chart.fig.set_size_inches(12,11)
g.fig.set_size_inches(12,11)

# A joint plot has 3 Axes, the first one called ax_joint 
# is the one we want to draw our court onto and adjust some other settings
ax = g.ax_joint
draw_court(ax)

# Adjust the axis limits and orientation of the plot in order
# to plot half court, with the hoop by the top of the plot
ax.set_xlim(-250,250)
ax.set_ylim(422.5, -47.5)

# Get rid of axis labels and tick marks
ax.set_xlabel('')
ax.set_ylabel('')
ax.tick_params(labelbottom='off', labelleft='off')

# Add a title
ax.set_title('Kobe Bryant FGA All seasons', 
             y=1.2, fontsize=14)

# Add Data Source 
ax.text(-250,445,'Data Source: Kaggle.com',
        fontsize=12)

plt.show()

In [ ]:
# create our jointplot

# get our colormap for the main kde plot
# Note we can extract a color from cmap to use for 
# the plots that lie on the side and top axes
cmap=plt.cm.gist_heat_r

# n_levels sets the number of contour lines for the main kde plot
joint_shot_chart = sns.jointplot(rawtrain['loc_x'],rawtrain['loc_y'], stat_func=None,
                                 kind='kde', space=0, color=cmap(0.1),
                                 cmap=cmap, n_levels=50)

joint_shot_chart.fig.set_size_inches(12,11)

# A joint plot has 3 Axes, the first one called ax_joint 
# is the one we want to draw our court onto and adjust some other settings
ax = joint_shot_chart.ax_joint
draw_court(ax)

# Adjust the axis limits and orientation of the plot in order
# to plot half court, with the hoop by the top of the plot
ax.set_xlim(-250,250)
ax.set_ylim(422.5, -47.5)

# Get rid of axis labels and tick marks
ax.set_xlabel('')
ax.set_ylabel('')
ax.tick_params(labelbottom='off', labelleft='off')


# Add a title
ax.set_title('Kobe Bryant FGA All seasons', 
             y=1.2, fontsize=14)

# Add Data Source 
ax.text(-250,445,'Data Source: Kaggle.com',
        fontsize=12)

In [ ]:
#Creation of Raw remaining time

raw['remaining time']=raw['minutes_remaining']* 60 +raw['seconds_remaining']
raw['season'].unique()

In [ ]:
raw['season'] = raw['season'].apply(lambda x: int(x.split('-')[1]) )
raw['season'].unique()

In [ ]:
#team id + team Name (Not usefull)
print(rawtrain['team_id'].unique())
print(rawtrain['team_name'].unique())
print(rawtrain['action_type'].unique())
print(rawtrain['combined_shot_type'].unique())
print(rawtrain['shot_type'].unique())

In [ ]:
drops = ['shot_id', 'team_id', 'team_name', 'shot_zone_area', 'shot_zone_range', 'shot_zone_basic', \
         'matchup', 'lon', 'lat', 'seconds_remaining', 'minutes_remaining', \
         'shot_distance', 'loc_x', 'loc_y', 'game_event_id', 'game_id', 'game_date']
for drop in drops:
    raw= raw.drop(drop, 1)
    
categorical_vars = ['action_type', 'combined_shot_type', 'shot_type', 'opponent', 'period', 'season']
for var in categorical_vars:
    raw = pd.concat([raw, pd.get_dummies(raw[var], prefix=var)], 1)
    raw = raw.drop(var, 1)

In [ ]:
import scipy as sp
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

In [ ]:
df = raw[pd.notnull(raw['shot_made_flag'])]
submission = raw[pd.isnull(raw['shot_made_flag'])]
submission = submission.drop('shot_made_flag', 1)

train = df.drop('shot_made_flag', 1)
train_y = df['shot_made_flag']

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
import time


# find the best n_estimators for RandomForestClassifier
print('Finding best n_estimators for RandomForestClassifier...')
min_score = 100000
best_n = 0
scores_n = []
range_n = np.logspace(0,2,num=3).astype(int)
for n in range_n:
    print("the number of trees : {0}".format(n))
    t1 = time.time()
    
    rfc_score = 0.
    rfc = RandomForestClassifier(n_estimators=n)
    for train_k, test_k in KFold(len(train), n_folds=10, shuffle=True):
        rfc.fit(train.iloc[train_k], train_y.iloc[train_k])
        #rfc_score += rfc.score(train.iloc[test_k], train_y.iloc[test_k])/10
        pred = rfc.predict(train.iloc[test_k])
        rfc_score += logloss(train_y.iloc[test_k], pred) / 10
    scores_n.append(rfc_score)
    if rfc_score < min_score:
        min_score = rfc_score
        best_n = n
        
    t2 = time.time()
    print('Done processing {0} trees ({1:.3f}sec)'.format(n, t2-t1))
print(best_n, min_score)


# find best max_depth for RandomForestClassifier
print('Finding best max_depth for RandomForestClassifier...')
min_score = 100000
best_m = 0
scores_m = []
range_m = np.logspace(0,2,num=3).astype(int)
for m in range_m:
    print("the max depth : {0}".format(m))
    t1 = time.time()
    
    rfc_score = 0.
    rfc = RandomForestClassifier(max_depth=m, n_estimators=best_n)
    for train_k, test_k in KFold(len(train), n_folds=10, shuffle=True):
        rfc.fit(train.iloc[train_k], train_y.iloc[train_k])
        #rfc_score += rfc.score(train.iloc[test_k], train_y.iloc[test_k])/10
        pred = rfc.predict(train.iloc[test_k])
        rfc_score += logloss(train_y.iloc[test_k], pred) / 10
    scores_m.append(rfc_score)
    if rfc_score < min_score:
        min_score = rfc_score
        best_m = m
    
    t2 = time.time()
    print('Done processing {0} trees ({1:.3f}sec)'.format(m, t2-t1))
print(best_m, min_score)

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(range_n, scores_n)
plt.ylabel('score')
plt.xlabel('number of trees')

plt.subplot(122)
plt.plot(range_m, scores_m)
plt.ylabel('score')
plt.xlabel('max depth')

In [ ]:
model = RandomForestClassifier(n_estimators=best_n, max_depth=best_m)
model.fit(train, train_y)
pred = model.predict_proba(submission)

sub = pd.read_csv("../input/sample_submission.csv")
sub['shot_made_flag'] = pred
sub.to_csv("submission.csv", index=False)